# Part Two

In [1]:
from part_one import *
from collections import defaultdict 



def fictitious_play(num_batt: int, 
                    att_res: int, 
                    def_res: int, 
                    att_prof: [(float, [bool])], 
                    def_prof: [(float, [bool])], 
                    batt_values: [int],
                    t = 100
                    ):
    asserts(num_batt, att_res, def_res, att_prof, batt_values)
    asserts(num_batt, att_res, def_res, def_prof, batt_values)  
    for _, att_strat in att_prof:
        assert len(att_strat) == num_batt
        assert sum(att_strat) == att_res
    for _, def_strat in def_prof:
        assert len(def_strat) == num_batt
        assert sum(def_strat) == def_res
    
    ap = defaultdict(lambda: 0)
    dp = defaultdict(lambda: 0)
    
    
    for prob, att_strat in att_prof:
        ap[stratToInt(att_strat)] += prob
    
    for prob, def_strat in def_prof:
        dp[stratToInt(def_strat)] += prob
    
    
 
    attacker_profs = [ap]
    defender_profs = [dp]
    
    
    for i in range(1, t): # first round is just choosing strategies
        unpack_att = [(prob, intToStrat(num_batt, strat)) for strat, prob in ap.items()]
        unpack_def = [(prob, intToStrat(num_batt, strat)) for strat, prob in dp.items()]
        
        att_counter = generate_perfect_attacker(num_batt, att_res, def_res, unpack_def, batt_values)
        def_counter = generate_perfect_defender(num_batt, att_res, def_res, unpack_att, batt_values)
        
        new_ap = defaultdict(lambda: 0)
        for strat, prob in ap.items():
            new_ap[strat] = (i / (i + 1)) * prob
        new_ap[stratToInt(att_counter)] += 1 / (i + 1)
        
        new_dp = defaultdict(lambda: 0)
        for strat, prob in dp.items():
            new_dp[strat] = (i / (i + 1)) * prob
        new_dp[stratToInt(def_counter)] += 1 / (i + 1)
        attacker_profs.append(new_ap)
        defender_profs.append(new_dp)
        ap = new_ap
        dp = new_dp
    return attacker_profs, defender_profs
    

In [9]:
att_prof = [(1/3, [1, 0, 0]), (1/3, [0, 1, 0]), (1/3, [0, 0, 1])]
def_prof = [(1/3, [1, 1, 0]), (1/3, [0, 1, 1]), (1/3, [1, 0, 1])]
# att_prof = [(1, [1, 0, 0])]
# def_prof = [(1, [0, 1, 1])]

attacker_profs, defender_profs = fictitious_play(3, 1, 2, att_prof, def_prof, [1, 100, 10000], 10000)

for a, d in list(zip(attacker_profs, defender_profs))[-10:]:
    print(dict(a))
    print(dict(d))
    print()

{1: 0.3026056784439335, 2: 0.3637607179795142, 4: 0.3336336035765515}
{3: 3.336336035765508e-05, 6: 0.9930270576852516, 5: 0.0069395789543922485}

{1: 0.3026754737123038, 2: 0.3637243127835595, 4: 0.3336002135041359}
{3: 3.3360021350413515e-05, 6: 0.993027755537765, 5: 0.006938884440886004}

{1: 0.30274525501184224, 2: 0.36368791487374424, 4: 0.3335668301144126}
{3: 3.335668301144119e-05, 6: 0.9930284532506102, 5: 0.006938190066379761}

{1: 0.30281502234674196, 2: 0.3636515242478814, 4: 0.33353345340537577}
{3: 3.335334534053751e-05, 6: 0.993029150823829, 5: 0.006937495830831795}

{1: 0.30288477572119454, 2: 0.36361514090378455, 4: 0.3335000833750201}
{3: 3.335000833750194e-05, 6: 0.9930298482574634, 5: 0.006936801734200396}

{1: 0.3029545151393897, 2: 0.3635787648392684, 4: 0.3334667200213411}
{3: 3.334667200213404e-05, 6: 0.9930305455515553, 5: 0.006936107776443873}

{1: 0.30302424060551564, 2: 0.36354239605214833, 4: 0.33343336334233525}
{3: 3.3343336334233456e-05, 6: 0.993031242706

In [10]:
# def_prof = [(.5, [1, 0, 1]), (.5, [0, 1, 1])]

def_prof = [(1, [0, 1, 1])]

generate_perfect_attacker(3, 1, 2, def_prof, [1, 2, 3])

[True, False, False]